In [6]:
import os
import pandas as pd
import humanfriendly
import numpy as np
from pathlib import Path

#WORKDIR=Path(os.environ["WORKDIR"])

def parse_size(values):
    size = np.zeros(values.shape, dtype=int)
    for j, val in enumerate(values):
        size[j] = humanfriendly.parse_size(val)
    return size

fn = "csvs/bitround_vcz_inspect.csv"
df_zarr= pd.read_csv(fn)
df_zarr["stored_bytes"] = parse_size(df_zarr.stored.values)
df_zarr.sort_values("stored_bytes", ascending=False, inplace=True)
df_zarr

,name,dtype,stored,size,ratio,nchunks,chunk_size,avg_chunk_stored,shape,chunk_shape,compressor,filters,stored_bytes
0,/call_LRR,float32,8.35 GiB,24.79 GiB,3.0000,726,34.96 MiB,11.78 MiB,"(10221, 651050)","(1000, 10000)","Blosc(cname='zstd', clevel=7, shuffle=NOSHUFFL...",[BitRound(keepbits=5)],8965744230
1,/call_BAF,float32,6.02 GiB,24.79 GiB,4.1000,726,34.96 MiB,8.49 MiB,"(10221, 651050)","(1000, 10000)","Blosc(cname='zstd', clevel=7, shuffle=NOSHUFFL...",[BitRound(keepbits=5)],6463925780
2,/call_GS,float32,961.37 MiB,24.79 GiB,26.0000,726,34.96 MiB,1.32 MiB,"(10221, 651050)","(1000, 10000)","Blosc(cname='zstd', clevel=7, shuffle=NOSHUFFL...",[BitRound(keepbits=5)],1008069509
3,/call_genotype,int8,657.21 MiB,12.39 GiB,19.0000,726,17.48 MiB,926.97 KiB,"(10221, 651050, 2)","(1000, 10000, 2)","Blosc(cname='zstd', clevel=7, shuffle=BITSHUFF...",NaN,689134632
4,/call_genotype_mask,bool,79.48 MiB,12.39 GiB,160.0000,726,17.48 MiB,112.1 KiB,"(10221, 651050, 2)","(1000, 10000, 2)","Blosc(cname='zstd', clevel=7, shuffle=BITSHUFF...",NaN,83340820
5,/sample_id,object,2.93 MiB,4.97 MiB,1.7000,66,77.07 KiB,45.44 KiB,"(651050,)","(10000,)","Blosc(cname='zstd', clevel=7, shuffle=SHUFFLE,...",[VLenUTF8()],3072327
6,/call_genotype_phased,bool,644.72 KiB,6.2 GiB,10000.0000,726,8.74 MiB,909 bytes,"(10221, 651050)","(1000, 10000)","Blosc(cname='zstd', clevel=7, shuffle=BITSHUFF...",NaN,660193
7,/variant_allele,object,57.45 KiB,159.7 KiB,2.8000,11,14.52 KiB,5.22 KiB,"(10221, 2)","(1000, 2)","Blosc(cname='zstd', clevel=7, shuffle=NOSHUFFL...",[VLenUTF8()],58828
8,/variant_filter,bool,48.97 KiB,9.98 KiB,0.2000,11,929.1818181818181 bytes,4.45 KiB,"(10221, 1)","(1000, 1)","Blosc(cname='zstd', clevel=7, shuffle=BITSHUFF...",NaN,50145
9,/variant_id,object,48.11 KiB,79.85 KiB,1.7000,11,7.26 KiB,4.37 KiB,"(10221,)","(1000,)","Blosc(cname='zstd', clevel=7, shuffle=NOSHUFFL...",[VLenUTF8()],49264


In [7]:
total = df_zarr.stored_bytes.sum()

humanfriendly.format_size(total, binary=True)

'16.03 GiB'

In [4]:
df_zarr.nchunks.sum()

4514

In [18]:
df_display_table = pd.DataFrame({
    "Field":df_zarr.name,
    "type": df_zarr.dtype,
    "storage":df_zarr.stored,
    "compress": df_zarr.ratio,
    "percentage": df_zarr.stored_bytes / total})
threshold = 0.001 / 100 # 0.001% 
df_display_table = df_display_table[df_display_table.percentage >= threshold].copy()
df_display_table.sort_values("percentage", ascending=False, inplace=True)
df_display_table["percentage"] = df_display_table["percentage"].map('{:.2%}'.format)
df_display_table["compress"] = df_display_table["compress"].map('{:.1f}'.format)
df_display_table

,Field,type,storage,compress,percentage
0,/call_LRR,float32,8.35 GiB,3.0,52.08%
1,/call_BAF,float32,6.02 GiB,4.1,37.55%
2,/call_GS,float32,961.37 MiB,26.0,5.86%
3,/call_genotype,int8,657.21 MiB,19.0,4.00%
4,/call_genotype_mask,bool,79.48 MiB,160.0,0.48%
5,/sample_id,object,2.93 MiB,1.7,0.02%
6,/call_genotype_phased,bool,644.72 KiB,10000.0,0.00%


In [21]:
print(df_display_table.to_latex(index=False, escape=True).replace("object", "str"))

\begin{tabular}{lllll}
\toprule
Field & type & storage & compress & percentage \\
\midrule
/call\_LRR & float32 & 8.35 GiB & 3.0 & 52.08\% \\
/call\_BAF & float32 & 6.02 GiB & 4.1 & 37.55\% \\
/call\_GS & float32 & 961.37 MiB & 26.0 & 5.86\% \\
/call\_genotype & int8 & 657.21 MiB & 19.0 & 4.00\% \\
/call\_genotype\_mask & bool & 79.48 MiB & 160.0 & 0.48\% \\
/sample\_id & str & 2.93 MiB & 1.7 & 0.02\% \\
/call\_genotype\_phased & bool & 644.72 KiB & 10000.0 & 0.00\% \\
\bottomrule
\end{tabular}



# Data for MAE

In [26]:
import io
data = """
|field|filter|mae|
| call_LRR | bitround | 0.0006934664561413229 |
| call_BAF | bitround | 0.0007864997605793178 |
| call_GS | bitround | 0.003377074608579278 |
| call_LRR | quantize | 1.907108071463881e-06 |
| call_BAF | quantize | 1.1768814829338226e-06 |
| call_GS | quantize | 1.8836773278962937e-06 |
"""
df = pd.read_csv(io.StringIO(data), sep="|")[["field", "filter", "mae"]]
pd.options.display.float_format = '{:.2e}'.format
df

,field,filter,mae
0,call_LRR,bitround,6.93e-04
1,call_BAF,bitround,7.86e-04
2,call_GS,bitround,3.38e-03
3,call_LRR,quantize,1.91e-06
4,call_BAF,quantize,1.18e-06
5,call_GS,quantize,1.88e-06


In [20]:
df.mae

0    0.000693
1    0.000786
2    0.003377
3    0.000002
4    0.000001
5    0.000002
Name: mae, dtype: float64